<a href="https://colab.research.google.com/github/aoiktd/test/blob/main/get_gdrive_path_from_url.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#@title URLの入力フォーム
url = "https://docs.google.com/spreadsheets/d/1GSWg89UasukJ7ALPjTgrPu-w2XxKIGwYrzOWF953rAc/edit?resourcekey#gid=2118082525&range=A11"#@param {type:"string"}

## Google Drive（共有フォルダ）内のファイルパスを表示

In [8]:
############
## IMPORT ##
############
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
import os
import google.colab
import googleapiclient.discovery
import googleapiclient.http
from googleapiclient._auth import default_credentials
import pandas as pd
from apiclient import errors


#####################
## drive apiの認証 ##
#####################
creds,project=google.auth.default()
drive_service_v2 = build('drive', 'v2', credentials=creds)
drive_service_v3 = build('drive', 'v3', credentials=creds)


##############
## 各種関数 ##
##############
def get_id_from_url(input_url):
  return input_url.split('/')[5]

def get_folder_name(shared_drive_id, parent_id, child_id):
  results = drive_service_v3.files().list(
    fields='*',
    corpora="drive",
    driveId=shared_drive_id,
    supportsTeamDrives = True,
    includeItemsFromAllDrives = True,
    includeTeamDriveItems = True,
    supportsAllDrives=True,
    q=f"'{parent_id}' in parents "
  ).execute()
  items = results.get('files', [])

  for item in items:
    if item['id']==child_id:
      return item['name']

def get_parents_list(par_list):
  file_id = par_list[-1]
  try:
    parents = drive_service_v2.parents().list(fileId=file_id).execute()
    for parent in parents['items']:
      # print('File Id: %s' % parent['id'])
      par_list.append(parent['id'])
      return get_parents_list(par_list)
    else:
      return par_list
  except errors.HttpError:
    print('HTTPError: %s' % errors)

def main():
  # 共有ドライブのID, 名前による辞書作成
  drives = drive_service_v3.drives().list(pageSize=100).execute()
  share_drive_dict = {}
  for sdrive in drives['drives']:
    share_drive_dict[sdrive['id']] = sdrive['name']
  # display(share_drive_dict)

  # URLからIDを取得
  id = get_id_from_url(url)

  # IDをもとにルートフォルダまでのパス上のIDリスト作成
  path_list = get_parents_list([id])
  # 一番最後のIDが共有ドライブのID
  SHARED_DRIVE_ID = path_list[-1]

  # IDリストからフォルダ名を取得し、パスの作成
  path = ''
  for i in range(len(path_list)-1):
    name = get_folder_name(SHARED_DRIVE_ID, parent_id=path_list[i+1], child_id=path_list[i])
    path = '\\'+name+path

  # 表示
  return '\\'+share_drive_dict[SHARED_DRIVE_ID]+path


##########
## 実行 ##
##########
path = main()
# PC用のパス
print('G:\\共有ドライブ'+path)
# Colab用のパス
print('/content/drive/Shareddrives'+path.replace('\\', '/'))

G:\共有ドライブ\OperationArchitect\ミニ開発チーム\Design Doc: 事前稟議運用\事前稟議申請フォーム（回答）
/content/drive/Shareddrives/OperationArchitect/ミニ開発チーム/Design Doc: 事前稟議運用/事前稟議申請フォーム（回答）
